In [1]:
#import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
import numpy as np

In [2]:
teamPath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/teamData.csv'
playerPath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/playerData.csv'
outcomePath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/outcomeData.csv'
teamData = pd.read_csv(teamPath)
playerData = pd.read_csv(playerPath, engine='python')
outcomeData = pd.read_csv(outcomePath)

In [3]:
outcomeData = outcomeData.drop(["Start Time", "Box Score", "Notes"], axis=1)

In [4]:
outcomeData.loc[(outcomeData["OT?"].isna()),'OT?']= 0
outcomeData.loc[(outcomeData["OT?"] == "OT"),'OT?']= 1
outcomeData.loc[(outcomeData["OT?"] == "2OT"),'OT?']= 2
outcomeData.loc[(outcomeData["OT?"] == "3OT"),'OT?']= 3
outcomeData.loc[(outcomeData["OT?"] == "4OT"),'OT?']= 4

In [5]:
def redefineTeamNames(dataframe):
    dataframe = dataframe.replace('Seattle SuperSonics', 'Oklahoma City Thunder', regex=True)
    
    dataframe = dataframe.replace('New Orleans/Oklahoma City Hornets', 'New Orleans Pelicans', regex=True)

    dataframe = dataframe.replace('New Orleans Hornets', 'New Orleans Pelicans', regex=True)
    
    dataframe = dataframe.replace('Charlotte Bobcats', 'Charlotte Hornets', regex=True)
    
    dataframe = dataframe.replace('New Jersey Nets', 'Brooklyn Nets', regex=True)
    
#     dataframe = dataframe.replace('*', '', regex=True)

    return dataframe

outcomeData = redefineTeamNames(outcomeData)

In [6]:
def homeWin(dataframe):
    if dataframe['Winner'] == dataframe['Home']:
        return True
    else:
        return False

outcomeData['HomeWin'] = outcomeData.apply(homeWin, axis=1)

In [7]:
encoding = LabelEncoder()
encoding.fit(outcomeData['Home'].values)
outcomeData['Home'] = encoding.transform(outcomeData['Home'].values)
outcomeData['Visitor'] = encoding.transform(outcomeData['Visitor'].values)

In [53]:
featurePath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/DifferentialFeatureData.csv'
X = pd.read_csv(featurePath)
y = outcomeData["HomeWin"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
validation_data = (X_test, y_test)

In [59]:
from keras import Sequential
from keras.layers import Dense, Dropout

classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(64, activation='relu', kernel_initializer='random_normal', input_dim=113))
classifier.add(Dropout(.5))
#Second  Hidden Layer
classifier.add(Dense(64, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dropout(.5))
#Third  Hidden Layer
classifier.add(Dense(64, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dropout(.5))
#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

#Compiling the neural network
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [60]:
from keras.callbacks import EarlyStopping
#Fitting the data to the training dataset
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 10)
classifier.fit(X_train,y_train, validation_data = validation_data, batch_size=128, epochs=100, callbacks=[es])

Train on 14303 samples, validate on 6131 samples
Epoch 1/100
14303/14303 [==============================] - 2s 152us/step - loss: 0.6311 - accuracy: 0.6450 - val_loss: 0.5809 - val_accuracy: 0.6909
Epoch 2/100
14303/14303 [==============================] - 3s 179us/step - loss: 0.6032 - accuracy: 0.6809 - val_loss: 0.5817 - val_accuracy: 0.6898
Epoch 3/100
14303/14303 [==============================] - 1s 89us/step - loss: 0.5984 - accuracy: 0.6814 - val_loss: 0.5818 - val_accuracy: 0.6987
Epoch 4/100
14303/14303 [==============================] - 1s 89us/step - loss: 0.5959 - accuracy: 0.6866 - val_loss: 0.5808 - val_accuracy: 0.6983
Epoch 5/100
14303/14303 [==============================] - 1s 77us/step - loss: 0.5939 - accuracy: 0.6838 - val_loss: 0.5766 - val_accuracy: 0.6974
Epoch 6/100
14303/14303 [==============================] - 1s 97us/step - loss: 0.5942 - accuracy: 0.6885 - val_loss: 0.5831 - val_accuracy: 0.6992
Epoch 7/100
14303/14303 [==============================] - 1s

In [61]:
#eval_model=classifier.evaluate(X_train, y_train)
#eval_model
eval_model=classifier.evaluate(X_test, y_test)
eval_model

6131/6131 [==============================] - 1s 96us/step


[0.5766993537751793, 0.7026585936546326]

In [45]:
from sklearn.metrics import confusion_matrix

y_pred=classifier.predict(X_test)
y_pred =(y_pred> 0.6)


[[0.20229256]
 [0.60726887]
 [0.51420593]
 ...
 [0.8662132 ]
 [0.53903145]
 [0.8276886 ]]


In [43]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1483  990]
 [1106 2552]]


In [33]:
(1042+3020)/(3020+1042+1431+638)

0.6625346599249714